In [1]:
import lightgbm as lgb
import os
import pandas as pd
import numpy as np
import random
import gc
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split

np.random.seed(2019)
random.seed(2019)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 280)
pd.set_option('display.max_colwidth', 150)
data_path = '/data/workspace/kimi/tencent_ads/2020/dataset'

In [2]:
train_df =pd.read_pickle('train1.pkl')
train_df.replace("\\N",-1,inplace=True)
train_df=train_df.astype(float,inplace=True)
train_df['age']  = train_df['age'] -1
train_df['gender']  = train_df['gender'] -1

print(train_df)

         user_id  active_days  click_times_total  age  gender  tfidf_0  tfidf_1  tfidf_2  tfidf_3  tfidf_4  tfidf_5  tfidf_6   tfidf_7  tfidf_8   tfidf_9  tfidf_10  tfidf_11  tfidf_12  tfidf_13  tfidf_14  tfidf_15  tfidf_16  tfidf_17  tfidf_18  tfidf_19  tfidf_20  tfidf_21  \
0            1.0         10.0               14.0  3.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  0.303336      0.0  0.000000  0.000000       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
1            2.0         28.0               46.0  9.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  0.000000      0.0  0.000000  0.000000       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2            3.0         23.0               30.0  6.0     1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  0.000000      0.0  0.000000  0.000000      

In [3]:
final_train_x_df = train_df.drop(['age','user_id','gender'], axis=1)
#final_train_x_df = train_df.drop(['age','user_id','gender','active_days'], axis=1)
final_train_y_df = train_df['gender']
X_train, X_test, y_train, y_test = train_test_split(final_train_x_df, final_train_y_df, test_size=0.20, random_state=42)
#train_data_show_df =train_df.drop(['active_days', 'click_times_total','max_clicked_industry_cnt','clicked_industry','clicked_advertiser','max_clicked_advertiser_cnt','max_clicked_industry_ratio','max_clicked_advertiser_ratio'], axis=1)
#print(X_train)
num_normal_features = ['_clicks_max_click_cnt','_max_clicked_ratio','_clicks_min_click_cnt','_min_clicked_ratio','_clicks_len','_clicks_mean','_clicks_median','_clicks_std']
num_date_features  = [ '_clicks_max_click_cnt', '_clicks_min_click_cnt','_clicks_len','_clicks_mean','_clicks_median','_clicks_std']
num_features = ['click_times_total'] +\
                [f'date{i}'  for i in num_date_features] + \
                [f'wday{i}'  for i in num_date_features] + \
                [f'month{i}'  for i in num_date_features] + \
                 [f'product_id{i}'  for i in num_normal_features] + \
                 [f'product_category{i}'  for i in num_normal_features] + \
                [f'industry{i}'  for i in num_normal_features] + \
                [f'advertiser_id{i}'  for i in num_normal_features]

#print(num_features)

c_features = ['industry_clicks_max_click','industry_clicks_min_click',
              'advertiser_id_clicks_max_click','advertiser_id_clicks_min_click',
              'product_id_clicks_max_click','product_id_clicks_min_click',
              'product_category_clicks_max_click','product_category_clicks_min_click',
             ]
features= num_features + c_features
features= [f"tfidf_{i}" for i in range(317)] +['active_days','click_times_total']
#train_data = lgb.Dataset(final_train_x_df, label=final_train_y_df, feature_name=[   'max_clicked_industry', 'max_clicked_advertiser_id' ], categorical_feature=['max_clicked_industry','max_clicked_advertiser_id'])
#train_data = lgb.Dataset(X_train, label=y_train, feature_name=features, categorical_feature=c_features,free_raw_data=False)
train_data = lgb.Dataset(X_train, label=y_train, feature_name=features,free_raw_data=False)

eval_data = lgb.Dataset(X_test, label=y_test, feature_name=features, categorical_feature=c_features,free_raw_data=False,reference=train_data)


In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'softmax',
    'num_class':2,
    'metric': 'multi_error',
    'num_leaves': 256,
    'learning_rate': 0.03,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.7,
    'bagging_freq': 5,
    'verbose': 1
}
print('Starting training...')
# train

gbm = lgb.train(params,
                train_data,
                valid_sets=[train_data,eval_data],
                num_boost_round = 1000,
                early_stopping_rounds=100
               )
                #early_stopping_rounds=5)

print('Feature importances:', list(gbm.feature_importance()))

Starting training...


/home/ubuntu/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[1]	training's multi_error: 0.330383	valid_1's multi_error: 0.330633
Training until validation scores don't improve for 100 rounds.
[2]	training's multi_error: 0.330383	valid_1's multi_error: 0.330633
[3]	training's multi_error: 0.330383	valid_1's multi_error: 0.330633
[4]	training's multi_error: 0.330383	valid_1's multi_error: 0.330633
[5]	training's multi_error: 0.330383	valid_1's multi_error: 0.330633
[6]	training's multi_error: 0.330383	valid_1's multi_error: 0.330633
[7]	training's multi_error: 0.330383	valid_1's multi_error: 0.330633
[8]	training's multi_error: 0.330383	valid_1's multi_error: 0.330633
[9]	training's multi_error: 0.330383	valid_1's multi_error: 0.330633
[10]	training's multi_error: 0.330383	valid_1's multi_error: 0.330633
[11]	training's multi_error: 0.3252	valid_1's multi_error: 0.325611
[12]	training's multi_error: 0.314326	valid_1's multi_error: 0.314967
[13]	training's multi_error: 0.300499	valid_1's multi_error: 0.301372
[14]	training's multi_error: 0.289757	

[118]	training's multi_error: 0.167442	valid_1's multi_error: 0.17335
[119]	training's multi_error: 0.167328	valid_1's multi_error: 0.173289
[120]	training's multi_error: 0.167214	valid_1's multi_error: 0.173189
[121]	training's multi_error: 0.167112	valid_1's multi_error: 0.173072
[122]	training's multi_error: 0.166943	valid_1's multi_error: 0.173006
[123]	training's multi_error: 0.166829	valid_1's multi_error: 0.172917
[124]	training's multi_error: 0.166721	valid_1's multi_error: 0.172883
[125]	training's multi_error: 0.166471	valid_1's multi_error: 0.172806
[126]	training's multi_error: 0.166358	valid_1's multi_error: 0.1726
[127]	training's multi_error: 0.166207	valid_1's multi_error: 0.172456
[128]	training's multi_error: 0.166076	valid_1's multi_error: 0.172394
[129]	training's multi_error: 0.165967	valid_1's multi_error: 0.172344
[130]	training's multi_error: 0.16586	valid_1's multi_error: 0.172217
[131]	training's multi_error: 0.165724	valid_1's multi_error: 0.172078
[132]	trai

[234]	training's multi_error: 0.157011	valid_1's multi_error: 0.166878
[235]	training's multi_error: 0.15696	valid_1's multi_error: 0.166878
[236]	training's multi_error: 0.156918	valid_1's multi_error: 0.166828
[237]	training's multi_error: 0.156867	valid_1's multi_error: 0.166806
[238]	training's multi_error: 0.156794	valid_1's multi_error: 0.166822
[239]	training's multi_error: 0.156736	valid_1's multi_error: 0.166783
[240]	training's multi_error: 0.156656	valid_1's multi_error: 0.166811
[241]	training's multi_error: 0.156586	valid_1's multi_error: 0.166789
[242]	training's multi_error: 0.156556	valid_1's multi_error: 0.1667
[243]	training's multi_error: 0.156537	valid_1's multi_error: 0.166789
[244]	training's multi_error: 0.156457	valid_1's multi_error: 0.166722
[245]	training's multi_error: 0.156401	valid_1's multi_error: 0.166717
[246]	training's multi_error: 0.156342	valid_1's multi_error: 0.166728
[247]	training's multi_error: 0.156269	valid_1's multi_error: 0.166578
[248]	tra

[350]	training's multi_error: 0.151326	valid_1's multi_error: 0.165167
[351]	training's multi_error: 0.151278	valid_1's multi_error: 0.165161
[352]	training's multi_error: 0.151224	valid_1's multi_error: 0.16525
[353]	training's multi_error: 0.151212	valid_1's multi_error: 0.165217
[354]	training's multi_error: 0.151157	valid_1's multi_error: 0.165239
[355]	training's multi_error: 0.151106	valid_1's multi_error: 0.165228
[356]	training's multi_error: 0.15109	valid_1's multi_error: 0.165233
[357]	training's multi_error: 0.151047	valid_1's multi_error: 0.165189
[358]	training's multi_error: 0.151006	valid_1's multi_error: 0.165178
[359]	training's multi_error: 0.150964	valid_1's multi_error: 0.165111
[360]	training's multi_error: 0.150939	valid_1's multi_error: 0.165156
[361]	training's multi_error: 0.150867	valid_1's multi_error: 0.165072
[362]	training's multi_error: 0.150808	valid_1's multi_error: 0.165089
[363]	training's multi_error: 0.150751	valid_1's multi_error: 0.165122
[364]	tr

[466]	training's multi_error: 0.146807	valid_1's multi_error: 0.164528
[467]	training's multi_error: 0.146779	valid_1's multi_error: 0.164544
[468]	training's multi_error: 0.146737	valid_1's multi_error: 0.164561
[469]	training's multi_error: 0.146711	valid_1's multi_error: 0.164528
[470]	training's multi_error: 0.146664	valid_1's multi_error: 0.164594
[471]	training's multi_error: 0.146611	valid_1's multi_error: 0.164594
[472]	training's multi_error: 0.146575	valid_1's multi_error: 0.164556
[473]	training's multi_error: 0.146544	valid_1's multi_error: 0.16455
[474]	training's multi_error: 0.146518	valid_1's multi_error: 0.164544
[475]	training's multi_error: 0.146476	valid_1's multi_error: 0.164544
[476]	training's multi_error: 0.14644	valid_1's multi_error: 0.164528
[477]	training's multi_error: 0.146429	valid_1's multi_error: 0.164567
[478]	training's multi_error: 0.146378	valid_1's multi_error: 0.164556
[479]	training's multi_error: 0.146367	valid_1's multi_error: 0.164544
[480]	tr

In [5]:
from sklearn.preprocessing import  OneHotEncoder

before_one_hot =  y_train.values.reshape([-1,1])
print(before_one_hot)
enc = OneHotEncoder()
enc.fit(before_one_hot)

one_hoted_y  = enc.transform(before_one_hot).toarray()
print(one_hoted_y.shape)


[[1.]
 [1.]
 [1.]
 ...
 [0.]
 [1.]
 [0.]]
(720000, 2)


/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [6]:
y_pred = gbm.predict(X_train)
for i in range(len(y_pred)):
        max_value=max(y_pred[i])
        for j in range(len(y_pred[i])):
            if max_value==y_pred[i][j]:
                y_pred[i][j]=1
            else:
                y_pred[i][j]=0
print(y_pred)
print(one_hoted_y)
classification_report(one_hoted_y, y_pred)
precision_score(one_hoted_y, y_pred,average='micro')

[[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]


0.8556194444444445

In [7]:
from sklearn.preprocessing import  OneHotEncoder

before_one_hot =  y_test.values.reshape([-1,1])
print(before_one_hot)
enc = OneHotEncoder()
enc.fit(before_one_hot)

one_hoted_y  = enc.transform(before_one_hot).toarray()
print(one_hoted_y.shape)

[[0.]
 [0.]
 [1.]
 ...
 [0.]
 [0.]
 [0.]]
(180000, 2)


/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [8]:
y_pred = gbm.predict(X_test)
for i in range(len(y_pred)):
        max_value=max(y_pred[i])
        for j in range(len(y_pred[i])):
            if max_value==y_pred[i][j]:
                y_pred[i][j]=1
            else:
                y_pred[i][j]=0
print(y_pred)
print(one_hoted_y)
classification_report(one_hoted_y, y_pred)
precision_score(one_hoted_y, y_pred,average='micro')

[[1. 0.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[1. 0.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


0.8354111111111111